Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [1]:
#@title # Setup
FixNotImplementedError = False # @param {type:"boolean"}
if(FixNotImplementedError):
  import locale
  locale.getpreferredencoding = lambda: "UTF-8"

!git clone https://github.com/152334H/tortoise-tts-fast
%cd tortoise-tts-fast
!pip3 install -r requirements.txt --no-deps
!pip3 install -e .
!pip3 install git+https://github.com/152334H/BigVGAN.git
#This solves the error "RuntimeError: Error(s) in loading state_dict for UnifiedVoice:"
!pip install transformers==4.29.2 pydub
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

from IPython.display import Audio, display
from pydub import AudioSegment

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

Cloning into 'tortoise-tts-fast'...
remote: Enumerating objects: 2314, done.
remote: Total 2314 (delta 0), reused 0 (delta 0), pack-reused 2314
Receiving objects: 100% (2314/2314), 105.06 MiB | 23.16 MiB/s, done.
Resolving deltas: 100% (1067/1067), done.
Updating files: 100% (697/697), done.
/content/tortoise-tts-fast
Ignoring backports-zoneinfo: markers 'python_version >= "3.8" and python_version < "3.9"' don't match your environment
ERROR: Can't verify hashes for these requirements because we don't have a way to hash version control repositories:
    bigvgan@ git+https://github.com/152334H/BigVGAN.git@HEAD from git+https://github.com/152334H/BigVGAN.git@HEAD (from -r requirements.txt (line 38))
Obtaining file:///content/tortoise-tts-fast
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/152334H/BigVG

(…)ritts-voxpopuli/resolve/main/config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

(…)0h/resolve/main/preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

(…)tacotron-symbols/resolve/main/vocab.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

(…)ols/resolve/main/special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Done.


Done.


Done.


Done.


In [2]:
#@title Display current voices available. (This cell is actually useless....)
#@markdown Tortoise will attempt to mimic voices you provide. It comes pre-packaged
#@markdown with some voices you might recognize.
#@markdown
#@markdown Let's list all the voices available. These are just some random clips I've gathered
#@markdown from the internet as well as a few voices from the training dataset.
#@markdown Feel free to add your own clips to the voices/ folder.
FixNotImplementedError = False # @param {type:"boolean"}
if(FixNotImplementedError):
  import locale
  locale.getpreferredencoding = lambda: "UTF-8"

%cd tortoise-tts-fast
%ls tortoise/voices
import IPython
display(Audio('tortoise/voices/tom/1.wav'))

[Errno 2] No such file or directory: 'tortoise-tts-fast'
/content/tortoise-tts-fast
angie/                emma/     lj/      rainbow/       train_daws/     train_kennard/
applejack/            freeman/  mol/     snakes/        train_dotrice/  train_lescault/
cond_latent_example/  geralt/   myself/  tim_reynolds/  train_dreams/   train_mouse/
daniel/               halle/    pat/     tom/           train_empire/   weaver/
deniro/               jlaw/     pat2/    train_atkins/  train_grace/    william/


In [16]:
#@title Change voice, or upload a new voice.
#@markdown
#@markdown Pick one of the voices from the output above, or choose custom, or random.
#@markdown
#@markdown Optionally, upload and use your own voice (probably what you came here to do). I recommend
#@markdown you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
#@markdown
#@markdown Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
#@markdown (Note: random voices can be prone to strange utterances)

voice = 'custom' #@param ["custom", "random", "angie", "applejack", "cond_latent_example", "daniel", "deniro", "emma", "freeman", "geralt", "halle", "jlaw", "lj", "mol", "myself", "pat", "pat2", "rainbow", "snakes", "tim_reynolds", "tom", "train_atkins", "train_daws", "train_dotrice", "train_dreams", "train_empire", "train_grace", "train_kennard", "train_lescault", "train_mouse", "weaver", "william"]
#@markdown If you've already uploaded a voice, but want to delete it and upload a new voice, click this here.
OverWritePastUpload = True #@param {type:"boolean"}

FixNotImplementedError = False # @param {type:"boolean"}
if(FixNotImplementedError):
  import locale
  locale.getpreferredencoding = lambda: "UTF-8"

if(voice == "custom"):
  import os
  from google.colab import files
  if(OverWritePastUpload):
    !rm -r /content/tortoise-tts-fast/tortoise/voices/custom
  custom_voice_folder = '/content/tortoise-tts-fast/tortoise/voices/custom'
  os.makedirs(custom_voice_folder)
  for i, file_data in enumerate(files.upload().values()):
    with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
      f.write(file_data)

if(not voice == "random"):
  voice_samples, conditioning_latents = load_voice(voice)

In [17]:
#@title Run this cell to ACTUALLY generate audio. You can change the text and run it again, even multiple times, without reinstalling tortoise, and you don't even have to re run the above cells.
#@markdown This is the text that will be spoken.
text = "Hello! This is Tortoise voice cloner! This is meant to be used for entertainment. Not for scams, or malicious intent. Please do not use this software for AI scams, because we are people with lives that you could destroy. Thank you and have fun!" #@param {type:"string"}
#@markdown Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast", "standard", "high_quality (default)"}. See docs in api.py
preset = "high_quality" #@param ["ultra_fast", "fast", "standard", "high_quality"]
#@markdown This is the new thing I added. The ability to type more sentences. I HIGHLY reccomend checking this box.
SplitSentences = True #@param {type:"boolean"}





#This converts the text into a format that Tortoise can use, by replacing stuff that tortoise doesn't know how to deal with.
text = text.replace("(",".")
text = text.replace(")",".")
text = text.replace(":.","...")
text = text.replace(".:","...")

text = text.replace("GB","Gigabyte")
text = text.replace("MB","Megabyte")
text = text.replace("KB","Kilobyte")
text = text.replace(" - "," dash ")
text = text.replace("-"," dash ")
text = text.replace(" + "," plus ")
text = text.replace("+"," plus ")
text = text.replace(" / "," slash ")
text = text.replace("/"," slash ")
text = text.replace(" _ "," underscore ")
text = text.replace("_"," underscore ")
text = text.replace("GPT","Jee Pee Tee")
text = text.replace("CPU","See Pee You")
text = text.replace("GPU","Jee Pee You")
text = text.replace("FBI","Eff Bee I")
text = text.replace("CDC","See Dee See")
text = text.replace("NIH","Enn I Aitch")
text = text.replace("MPS","Emm Pee Ess")
text = text.replace("TTS","Tee Tee Ess")
text = text.replace("AI","Aey I")

repeat = 0
while(repeat < 10):
  repeat2 = 0
  while(repeat2 < 10):
    text = text.replace(f"{repeat}.{repeat2}",f"{repeat} point {repeat2}")
    repeat2 = repeat2 + 1
  repeat = repeat + 1




#Ignore this, this only makes merging audio files easier. It's a custom made function
def mergeAudioFiles(inputFiles, outputFile):
  #but if you insist on finding out how this works, I will explain it.
  #This selects the first audio file from the list, and adds it to combined_sounds. which is currently empty
  combined_sounds = AudioSegment.from_wav(f"{inputFiles[0]}")
  #This is a tracker variable to track which file we are on
  fileNumber = 1
  #This is a loop which will go through every single file in the input files
  while(fileNumber < len(inputFiles)):
    #This adds the next sound file in the list to the combined_sounds variable
    combined_sounds = combined_sounds + AudioSegment.from_wav(f"{inputFiles[fileNumber]}")
    #This increases the file in position by one, and repeats the loop
    fileNumber = fileNumber + 1
  #This exports the combined sounds
  combined_sounds.export(f"{outputFile}", format="wav")
  #This tells the user where the output file was saved to.
  print(f'Output file saved to {os.getcwd()}/{outputFile}')
###################################################################################




#Loads it and send it through Tortoise.
if(not SplitSentences):
  if((not voice == "custom") and (not voice == "random")):
    torchaudio.save(f'generated-{voice}.wav', tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  if(voice == "random"):
    torchaudio.save(f'generated-{voice}.wav', tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  # Generate speech with the custom voice.
  if(voice == "custom"):
    torchaudio.save(f'generated-{voice}.wav', tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'generated-{voice}.wav', autoplay=True))

if(SplitSentences):
  #Since Tortoise is really bad on long audio, this will split the text up into multiple sentences.
  sentences = []
  temporarySentence = ""
  multipleDotsTogether = False
  characterNumber = 0
  while(characterNumber < len(text)):
    temporarySentence = temporarySentence + text[characterNumber]
    if(characterNumber < len(text) -1):
      if(text[characterNumber] == "." and text[characterNumber+1] == "."):
        multipleDotsTogether = True
    if(characterNumber < len(text) -1):
      if(text[characterNumber] == "!" and text[characterNumber+1] == "!"):
        multipleDotsTogether = True
    if(characterNumber < len(text) -1):
      if(text[characterNumber] == "?" and text[characterNumber+1] == "?"):
        multipleDotsTogether = True
    if((text[characterNumber] == ":" or text[characterNumber] == "." or text[characterNumber] == "!" or text[characterNumber] == "?" or text[characterNumber] == "," or text[characterNumber] == ";") and (not multipleDotsTogether)):
      sentences.append(temporarySentence.strip())
      temporarySentence = ""
    multipleDotsTogether = False
    characterNumber = characterNumber + 1
  if(len(temporarySentence.strip()) > 1):
    sentences.append(temporarySentence.strip())



  print("Tortoise split the text into multiple sentences. It will generate one at a time, then combine them together in a single file for you.")
  print("Please be patient, this will take a LONG time. Literally....")
  print("Here are the sentences Tortoise will generate in order:")
  print()
  for sentence in sentences:
    print(f'"{sentence}"')
  print()
  print()
  #This actually generates the audio.
  if((not voice == "custom") and (not voice == "random")):
    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'generated-{voice}-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'generated-{voice}-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'generated-{voice}-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'generated-{voice}.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  if(voice == "random"):
    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'generated-{voice}-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=None, conditioning_latents=None, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'generated-{voice}-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'generated-{voice}-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'generated-{voice}.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  if(voice == "custom"):
    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'generated-{voice}-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'generated-{voice}-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'generated-{voice}-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'generated-{voice}.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'generated-{voice}.wav', autoplay=True))


Output hidden; open in https://colab.research.google.com to view.

In [19]:
#@title Combine voices into one.
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:

RunCombo = False #@param {type:"boolean"}
voice1 = "pat" # @param ["custom", "angie", "applejack", "cond_latent_example", "daniel", "deniro", "emma", "freeman", "geralt", "halle", "jlaw", "lj", "mol", "myself", "pat", "pat2", "rainbow", "snakes", "tim_reynolds", "tom", "train_atkins", "train_daws", "train_dotrice", "train_dreams", "train_empire", "train_grace", "train_kennard", "train_lescault", "train_mouse", "weaver", "william"] {allow-input: true}
voice2 = "william" # @param ["custom", "angie", "applejack", "cond_latent_example", "daniel", "deniro", "emma", "freeman", "geralt", "halle", "jlaw", "lj", "mol", "myself", "pat", "pat2", "rainbow", "snakes", "tim_reynolds", "tom", "train_atkins", "train_daws", "train_dotrice", "train_dreams", "train_empire", "train_grace", "train_kennard", "train_lescault", "train_mouse", "weaver", "william"] {allow-input: true}
text = "They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to."#@param {type:"string"}
preset = "high_quality" #@param ["ultra_fast", "fast", "standard", "high_quality"]
SplitSentences = True #@param {type:"boolean"}

def mergeAudioFiles(inputFiles, outputFile):
  #but if you insist on finding out how this works, I will explain it.
  #This selects the first audio file from the list, and adds it to combined_sounds. which is currently empty
  combined_sounds = AudioSegment.from_wav(f"{inputFiles[0]}")
  #This is a tracker variable to track which file we are on
  fileNumber = 1
  #This is a loop which will go through every single file in the input files
  while(fileNumber < len(inputFiles)):
    #This adds the next sound file in the list to the combined_sounds variable
    combined_sounds = combined_sounds + AudioSegment.from_wav(f"{inputFiles[fileNumber]}")
    #This increases the file in position by one, and repeats the loop
    fileNumber = fileNumber + 1
  #This exports the combined sounds
  combined_sounds.export(f"{outputFile}", format="wav")
  #This tells the user where the output file was saved to.
  print(f'Output file saved to {os.getcwd()}/{outputFile}')




if(RunCombo):
  if(not SplitSentences):
    voice_samples, conditioning_latents = load_voices([voice1, voice2])
    torchaudio.save('captain_kirkard.wav', tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'captain_kirkard.wav', autoplay=True))
  if(SplitSentences):
    voice_samples, conditioning_latents = load_voices([voice1, voice2])
    #Since Tortoise is really bad on long audio, this will split the text up into multiple sentences.
    sentences = []
    temporarySentence = ""
    multipleDotsTogether = False
    characterNumber = 0
    while(characterNumber < len(text)):
      temporarySentence = temporarySentence + text[characterNumber]
      if(characterNumber < len(text) -1):
        if(text[characterNumber] == "." and text[characterNumber+1] == "."):
          multipleDotsTogether = True
      if(characterNumber < len(text) -1):
        if(text[characterNumber] == "!" and text[characterNumber+1] == "!"):
          multipleDotsTogether = True
      if(characterNumber < len(text) -1):
        if(text[characterNumber] == "?" and text[characterNumber+1] == "?"):
          multipleDotsTogether = True
      if((text[characterNumber] == ":" or text[characterNumber] == "." or text[characterNumber] == "!" or text[characterNumber] == "?" or text[characterNumber] == "," or text[characterNumber] == ";") and (not multipleDotsTogether)):
        sentences.append(temporarySentence.strip())
        temporarySentence = ""
      multipleDotsTogether = False
      characterNumber = characterNumber + 1
    if(len(temporarySentence.strip()) > 1):
      sentences.append(temporarySentence.strip())

    print("Tortoise split the text into multiple sentences. It will generate one at a time, then combine them together in a single file for you.")
    print("Please be patient, this will take a LONG time. Literally....")
    print("Here are the sentences Tortoise will generate in order:")
    print()
    for sentence in sentences:
      print(f'"{sentence}"')
    print()
    print()

    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'captain_kirkard-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'captain_kirkard-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'captain_kirkard-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'captain_kirkard.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'captain_kirkard.wav', autoplay=True))


mode 0
Generating autoregressive samples..


100%|██████████| 1/1 [00:07<00:00,  7.79s/it]


Computing best candidates using CLVP


100%|██████████| 1/1 [00:00<00:00,  5.32it/s]


Transforming autoregressive outputs into audio..


  0%|          | 0/10 [00:00<?, ?it/s]